<a href="https://colab.research.google.com/github/raju1207/CarDekho-Data-Prediction/blob/main/Car_Resale_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# Data loading
df = pd.read_excel('kolkata_cars.xlsx')
df1 = pd.read_excel('jaipur_cars.xlsx')
df2 = pd.read_excel('delhi_cars.xlsx')
df3 = pd.read_excel('chennai_cars.xlsx')
df4 = pd.read_excel('bangalore_cars.xlsx')
df5 = pd.read_excel('hyderabad_cars.xlsx')

In [5]:
def car_overview_dict_creator(overview):
    overview_dict = json.loads(overview.replace("'", '"').replace('None', "1"))
    all_overviews = {}
    for dictionary in overview_dict['top']:
        all_overviews.update({dictionary['key']: dictionary['value']})
    return all_overviews

In [6]:
def car_feature_dict_creator(feature):
  all_features = {}
  feature_dict = json.loads(feature.replace("'", '"').replace('None', "1"))
  for dictionary in feature_dict['top']:
    all_features.update({dictionary['value']: True})
    for section in feature_dict.get('data', []):
      for item in section.get('list', []):
        all_features.update({item['value']: True})
  return all_features

In [7]:
def car_spec_dict_creator(spec):
    all_specs = {}
    specs_dict = json.loads(spec.replace("'", '"').replace('None', "null"))
    for dictionary in specs_dict.get('top', []):
      all_specs.update({dictionary['key']: dictionary['value']})
    # Extract nested specifications
    for dictionary in specs_dict.get('data', []):
      for item in dictionary.get('list', []):
        all_specs.update({item['key'] : item['value']})
    return all_specs

In [8]:
df.new_car_detail=df['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df_car_detail = pd.DataFrame(df.new_car_detail.to_list())

# to extract car overview
df.new_car_overview = df['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df_car_overview = pd.DataFrame(df.new_car_overview.to_list())

# to extract car feature
feature = pd.DataFrame(df.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df = df.merge(feature, left_index=True, right_index=True)
df['feature_dict'] = df['feature'].apply(car_feature_dict_creator)
feature_df = pd.json_normalize(df['feature_dict']).fillna(False).astype(int)

# extract car specs
df['new_car_specs'] = df['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df_car_specs = pd.DataFrame(df['new_car_specs'].tolist())

/tmp/ipython-input-8-2892571178.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df = pd.json_normalize(df['feature_dict']).fillna(False).astype(int)


In [9]:
df_kolkata=pd.concat([df_car_detail,df_car_overview,feature_df,df_car_specs],axis=1)
df_kolkata["city"]="Kolkata"
df_kolkata.to_csv("Kolkata_Data.csv",index=False)

In [10]:
df1.new_car_detail=df1['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df1_car_detail = pd.DataFrame(df1.new_car_detail.to_list())

# to extract car overview
df1.new_car_overview = df1['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df1_car_overview = pd.DataFrame(df1.new_car_overview.to_list())

# to extract car feature
feature = pd.DataFrame(df1.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df1 = df1.merge(feature, left_index=True, right_index=True)
df1['feature_dict'] = df1['feature'].apply(car_feature_dict_creator)
feature_df1 = pd.json_normalize(df1['feature_dict']).fillna(False).astype(int)

# extract car specs
df1['new_car_specs'] = df1['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df1_car_specs = pd.DataFrame(df1['new_car_specs'].tolist())

/tmp/ipython-input-10-49220469.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df1 = pd.json_normalize(df1['feature_dict']).fillna(False).astype(int)


In [20]:
df_jaipur = pd.concat([df1_car_detail, df1_car_overview, feature_df1, df1_car_specs], axis=1)
df_jaipur["city"] = "Jaipur"
df_jaipur.to_csv("Jaipur_Data.csv", index=False)

In [12]:
df2.new_car_detail=df2['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df2_car_detail = pd.DataFrame(df2.new_car_detail.to_list())

# to extract car overview
df2.new_car_overview = df2['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df2_car_overview = pd.DataFrame(df2.new_car_overview.to_list())

# to extract car feature
feature = pd.DataFrame(df2.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df2 = df2.merge(feature, left_index=True, right_index=True)
df2['feature_dict'] = df2['feature'].apply(car_feature_dict_creator)
feature_df2 = pd.json_normalize(df2['feature_dict']).fillna(False).astype(int)

# extract car specs
df2['new_car_specs'] = df2['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df2_car_specs = pd.DataFrame(df2['new_car_specs'].tolist())

/tmp/ipython-input-12-1371010580.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df2 = pd.json_normalize(df2['feature_dict']).fillna(False).astype(int)


In [21]:
df_delhi = pd.concat([df2_car_detail, df2_car_overview, feature_df2, df2_car_specs], axis=1)
df_delhi["city"] = "Delhi"
df_delhi.to_csv("Delhi_Data.csv", index=False)

In [14]:
df3.new_car_detail=df3['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df3_car_detail = pd.DataFrame(df3.new_car_detail.to_list())

# to extract car overview
df3.new_car_overview = df3['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df3_car_overview = pd.DataFrame(df3.new_car_overview.to_list())

# to extract car feature
feature = pd.DataFrame(df3.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df3 = df3.merge(feature, left_index=True, right_index=True)
df3['feature_dict'] = df3['feature'].apply(car_feature_dict_creator)
feature_df3 = pd.json_normalize(df3['feature_dict']).fillna(False).astype(int)

# extract car specs
df3['new_car_specs'] = df3['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df3_car_specs = pd.DataFrame(df3['new_car_specs'].tolist())

/tmp/ipython-input-14-2060700622.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df3 = pd.json_normalize(df3['feature_dict']).fillna(False).astype(int)


In [22]:
df_chennai = pd.concat([df3_car_detail, df3_car_overview, feature_df3, df3_car_specs], axis=1)
df_chennai["city"] = "Chennai"
df_chennai.to_csv("Chennai_Data.csv", index=False)

In [16]:
df4.new_car_detail = df4['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df4_car_detail = pd.DataFrame(df4.new_car_detail.to_list())

# to extract car overview
df4.new_car_overview = df4['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df4_car_overview = pd.DataFrame(df4.new_car_overview.to_list())

# to extract car feature
feature = pd.DataFrame(df4.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df4 = df4.merge(feature, left_index=True, right_index=True)
df4['feature_dict'] = df4['feature'].apply(car_feature_dict_creator)
feature_df4 = pd.json_normalize(df4['feature_dict']).fillna(False).astype(int)

# extract car specs
df4['new_car_specs'] = df4['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df4_car_specs = pd.DataFrame(df4['new_car_specs'].tolist())

/tmp/ipython-input-16-1505181291.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df4 = pd.json_normalize(df4['feature_dict']).fillna(False).astype(int)


In [24]:
df_Bangalore = pd.concat([df4_car_detail, df4_car_overview, feature_df4, df4_car_specs], axis=1)
df_Bangalore["city"] = "Bangalore"
df_Bangalore.to_csv("Bangalore_Data.csv", index=False)

In [18]:
df5.new_car_detail=df5['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df5_car_detail = pd.DataFrame(df5.new_car_detail.to_list())

# to extract car overview
df5.new_car_overview = df5['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df5_car_overview = pd.DataFrame(df5.new_car_overview.to_list())

# to extract car feature
feature = pd.DataFrame(df5.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df5 = df5.merge(feature, left_index=True, right_index=True)
df5['feature_dict'] = df5['feature'].apply(car_feature_dict_creator)
feature_df5 = pd.json_normalize(df5['feature_dict']).fillna(False).astype(int)

# extract car specs
df5['new_car_specs'] = df5['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df5_car_specs = pd.DataFrame(df5['new_car_specs'].tolist())

/tmp/ipython-input-18-3600266653.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df5 = pd.json_normalize(df5['feature_dict']).fillna(False).astype(int)


In [25]:
df_hyderabad = pd.concat([df5_car_detail, df5_car_overview, feature_df5, df5_car_specs], axis=1)
df_hyderabad["city"] = "Hyderabad"
df_hyderabad.to_csv("Hyderabad_Data.csv", index=False)